# <center style="background-color:#606F66;color: white; vertical-align: middle; padding:20px 5px;">Aerial Semantic Segmentation Drone </center>

## <center style="background-color:#EF6145;color: white; vertical-align: middle; padding:10px 5px;">Importing Libraries</center>

In [2]:
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import os
from PIL import Image

import imgaug as ia
from imgaug import augmenters

from functools import partial

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle


from RGB_Mask import RGB_to_Mask
from Augmentaion_Pipeline import  image_augmentation



# <center style="background-color:#54A57C;color: white; vertical-align: middle; padding:10px 5px;">Preprocessing</center>

In [3]:
#IMG properties
IMG_HEIGHT = 800
IMG_WIDTH  = 1200
IMG_CHANNELS = 3
Num_Classes = 23
reshape_dims = (IMG_HEIGHT,IMG_WIDTH)
combined=[Num_Classes,reshape_dims]# to pass as argument for map function
batch_size = 16

## <center style="background-color:#EF6145;color: white; vertical-align: middle; padding:10px 5px;">Image Path glob</center>

In [3]:
#Mask related paths
RGB_Mask_PNG_LIST = tf.io.gfile.glob("../Data/RGB_color_image_masks/*.png")
RGB_Mask_PNG_LIST.sort()

RGB_to_mask_path = '../Data/Mask'
Mask_aug_path    = '../Data/Mask_Augmented'

#orginal Re;lated paths
ORIGINAL_JPG_LIST = tf.io.gfile.glob('../Data/original_images/*.jpg')
ORIGINAL_JPG_LIST.sort()
Orginal_aug_path = '../Data/Orginal_Augmented'

read_csv = pd.read_csv('../Data/class_dict_seg.csv',index_col=False,skipinitialspace=True)



## <center style="background-color:#EF6145;color: white; vertical-align: middle; padding:10px 5px;">RGB to Mask</center>

In [4]:
RGB_to_Mask(read_csv,RGB_Mask_PNG_LIST,RGB_to_mask_path) #only one time run
Mask_PNG_LIST = tf.io.gfile.glob("../Data/Mask/*.png")
Mask_PNG_LIST.sort()

## <center style="background-color:#EF6145;color: white; vertical-align: middle; padding:10px 5px;">Augmentaion</center>

In [6]:
rot_90 = augmenters.Rot90(1,keep_size=False) #rot_90 = augmenters.Rot90(times) #time=1 =>90deg, time=2 =>180deg, time=3 =>270deg
rot_180 = augmenters.Rot90(2,keep_size=False) 
rot_270 = augmenters.Rot90(3,keep_size=False)
flip_hor = augmenters.Fliplr(1) 
fli_vert = augmenters.Flipud(1)

pipeline = [rot_90, rot_180, rot_270, flip_hor, fli_vert]


#image_augmentation(ORIGINAL_JPG_LIST,Orginal_aug_path,pipeline)#orginal augmentaion
image_augmentation(Mask_PNG_LIST,Mask_aug_path,pipeline)#mask augmentaion



In [ ]:


ORIGINAL_JPG_LIST_with_aug = ORIGINAL_JPG_LIST+tf.io.gfile.glob('./Data/Orginal_Augmented/*.jpg')
Mask_PNG_LIST_with_aug      = Mask_PNG_LIST+tf.io.gfile.glob('./Data/Mask_Augmented/*.png')


index= [os.path.splitext(filename)[0] for filename in os.listdir('./Data/original_images/')]
index_with_aug=index+[os.path.splitext(filename)[0] for filename in os.listdir('./Data/Orginal_Augmented/')]

ORIGINAL_JPG_LIST_with_aug.sort()
Mask_PNG_LIST_with_aug.sort()
index_with_aug.sort()

## <center style="background-color:#EF6145;color: white; vertical-align: middle; padding:10px 5px;">Creating Dataframe from paths</center>

In [ ]:
df=pd.DataFrame((index_with_aug,ORIGINAL_JPG_LIST_with_aug,Mask_PNG_LIST_with_aug)).T
df.rename(columns={
    0: 'index',
    1: 'Orginal',
    2: 'Segmented'},
          inplace=True)
df.set_index('index', inplace= True)
df.to_csv('DataFrame.csv')

df = shuffle(df)


In [ ]:
X_train,X_val = train_test_split(df, test_size=0.2, random_state=42)
X_val,X_test = train_test_split(X_val, test_size=0.25, random_state=42)

## <center style="background-color:#EF6145;color: white; vertical-align: middle; padding:10px 5px;">Convert dataframe to tensorflow BatchDataset</center>

In [ ]:
Train_DF= (tf.data.Dataset.from_tensor_slices((X_train['Orginal'].values,X_train['Segmented'].values)))

Eval_DF = (tf.data.Dataset.from_tensor_slices((X_val['Orginal'].values,X_val['Segmented'].values)))


2022-06-15 07:33:40.813593: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-06-15 07:33:40.813645: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (UB): /proc/driver/nvidia/version does not exist
2022-06-15 07:33:40.815369: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
Train_BatchDataset = (Train_DF.map(partial(decode_csv, combined))).batch(batch_size)
Eval_BatchDataset  = (Eval_DF.map(partial(decode_csv, combined))).batch(batch_size)


<br><br><br>


code to save


H=1

original_img=Image.open(ORIGINAL_JPG_LIST[H])
original_img= np.array(original_img)
mask_image = Image.open(Mask_PNG_LIST[H])
mask_image = np.array(mask_image)

orginal_aug=oh_shit(original_img,pipeline)
mask_aug=oh_shit(mask_image,pipeline)

m = 3

fig, axs = plt.subplots(1, 2, figsize=(25, 10), constrained_layout=True)

axs[0].imshow(orginal_aug[m])
axs[0].grid(False)


axs[1].imshow(saved_loaded)
axs[1].grid(False)
